# Chapter 11 Exercise

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [2]:
import tensorflow as tf

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [4]:
reset_graph()

# Condensed form of the above code
def dnn(X,y,n_hidden1,n_hidden2,n_outputs,initializer=None,seed=42,learning_rate=0.03):
    with tf.name_scope("dnn"):
        hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
        hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
        logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
        y_prob = tf.nn.softmax(logits)
    with tf.name_scope("loss"):
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")
    with tf.name_scope("train"):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        training_op = optimizer.minimize(loss)
        loss_summary = tf.summary.scalar('log_loss', loss)
    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    with tf.name_scope("init"):
        init = tf.global_variables_initializer()
    with tf.name_scope("save"):
        saver = tf.train.Saver()
    return y_prob, logits, loss, training_op, loss_summary, accuracy, init, saver

In [5]:
# For TensorBoard
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

In [6]:
# Graph for TensorBoard
# (not using an enhanced X, using X from before)
n_inputs = 28*28  # MNIST
n_layers = 5
n_hidden = 100
n_outputs = 28*28
logdir = log_dir("dnn11")

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

y_prob, logits, loss, training_op, loss_summary, accuracy, init, saver = dnn(X,y,n_layers,n_hidden,n_outputs)

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [7]:
n_epochs = 20
batch_size = 50

checkpoint_path = "/tmp/dnn11.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "./dnn11"

In [8]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [9]:
with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        # if the checkpoint file exists, restore the model and load the epoch number
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)
        
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./my_dnn11_final.ckpt")

0 Batch accuracy: 0.96 Val accuracy: 0.93
1 Batch accuracy: 0.94 Val accuracy: 0.9506
2 Batch accuracy: 0.96 Val accuracy: 0.9598
3 Batch accuracy: 0.96 Val accuracy: 0.9662
4 Batch accuracy: 0.98 Val accuracy: 0.969
5 Batch accuracy: 0.98 Val accuracy: 0.9668
6 Batch accuracy: 1.0 Val accuracy: 0.9756
7 Batch accuracy: 0.98 Val accuracy: 0.9756
8 Batch accuracy: 1.0 Val accuracy: 0.977
9 Batch accuracy: 0.98 Val accuracy: 0.9776
10 Batch accuracy: 0.98 Val accuracy: 0.9786
11 Batch accuracy: 1.0 Val accuracy: 0.9804
12 Batch accuracy: 0.98 Val accuracy: 0.9764
13 Batch accuracy: 1.0 Val accuracy: 0.9776
14 Batch accuracy: 1.0 Val accuracy: 0.9798
15 Batch accuracy: 1.0 Val accuracy: 0.9806
16 Batch accuracy: 1.0 Val accuracy: 0.9804
17 Batch accuracy: 1.0 Val accuracy: 0.9816
18 Batch accuracy: 1.0 Val accuracy: 0.9812
19 Batch accuracy: 1.0 Val accuracy: 0.9822


I changed the learning rate from 0.01 to 0.03 and got a higher accuracy.

In [10]:
with tf.Session() as sess:
    saver.restore(sess, "./my_dnn11_final.ckpt") # or better, use save_path
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [11]:
from tensorflow_graph_in_jupyter import show_graph

show_graph(tf.get_default_graph())